In [ ]:
from glove import Glove
from glove import Corpus

In [1]:
import pandas as pd
import numpy as np
import jieba
import re
import matplotlib.pyplot as plt
from gensim.models.word2vec import Word2Vec
from gensim.models import FastText
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

In [35]:
import tensorflow as tf

In [2]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import  pad_sequences
from keras.preprocessing import sequence
from keras.layers import Input, Dropout, Dense, BatchNormalization, Activation, concatenate, GlobalMaxPool1D,GRU, Embedding,Bidirectional, Flatten,LSTM, BatchNormalization,Conv1D,MaxPooling1D
from keras.layers import GlobalMaxPooling1D,Concatenate,Activation, Lambda,SimpleRNN,LSTM,GRU,Bidirectional,TimeDistributed
from keras.layers import concatenate, GlobalMaxPooling1D,SpatialDropout1D,GlobalAveragePooling1D,GlobalAveragePooling1D
from keras.layers.convolutional import Convolution1D
from keras import optimizers
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping
from keras import backend as K
from keras import regularizers
from keras.utils import np_utils
from keras.models import Sequential,Model
from keras.callbacks import ModelCheckpoint,EarlyStopping

D:\python\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


评价指标

In [3]:
def precision(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision
def recall(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall
def fbeta_score(y_true, y_pred, beta=1):
    if beta < 0:
        raise ValueError('The lowest choosable beta is zero (only precision).')
    if K.sum(K.round(K.clip(y_true, 0, 1))) == 0:
        return 0
    p = precision(y_true, y_pred)
    r = recall(y_true, y_pred)
    bb = beta ** 2
    fbeta_score = (1 + bb) * (p * r) / (bb * p + r + K.epsilon())
    return fbeta_score
def fmeasure(y_true, y_pred):
    return fbeta_score(y_true, y_pred, beta=1)

模型结构

In [4]:
def cnn(_filters,_kernel,_drop):
    comment_seq_pre = Input(shape=[maxlen],name='Discuss_seq_pre')
    emb_comment1 =Embedding(len(word_index) + 1, embed_size,weights=[embedding_matrix])(comment_seq_pre)
    con1 = Conv1D(filters=_filters, kernel_size=_kernel, padding='same', activation='relu')(emb_comment1)
    maxp1 = MaxPooling1D(pool_size=2)(con1)
    fla = Flatten()(maxp1)
    drop = Dropout(_drop)(fla)
    output = Dense(3,activation='softmax')(drop)
    model = Model(inputs=[comment_seq_pre],outputs=[output])
    adam = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
    model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["mae", fmeasure])
    return model

def cnn_gru(_filters, _kernel, _neurons, _drop):
    comment_seq_pre = Input(shape=[maxlen])
    emb_comment = Embedding(len(word_index) + 1,embed_size,weights=[embedding_matrix],input_length=maxlen)(comment_seq_pre)
    con1 = Conv1D(filters=_filters, kernel_size=_kernel, padding='same', activation='relu')(emb_comment)
    maxp1 = MaxPooling1D(pool_size=2)(con1)
    gru = GRU(units=_neurons, activation='tanh')(maxp1)
    drop1 = Dropout(_drop)(gru)
    output = Dense(3, activation="softmax")(drop1)
    adam = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
    model = Model(inputs=comment_seq_pre, outputs=output)
    model.compile(loss='categorical_crossentropy',optimizer=adam,metrics=["mae", fmeasure])
    return model

def cnn_lstm(_filters, _kernel, _neurons, _drop):
    comment_seq_pre = Input(shape=[maxlen])
    emb_comment1 =Embedding(len(word_index) + 1, embed_size,weights=[embedding_matrix])(comment_seq_pre)
    con = Conv1D(filters=_filters, kernel_size=_kernel, padding='same', activation='relu')(emb_comment1)
    maxp = MaxPooling1D(pool_size=2)(con)
    lst_1=Dropout(_drop)(LSTM(units=_neurons, activation='tanh')(maxp))
    output = Dense(3,activation='softmax')(lst_1)
    model = Model(inputs=[comment_seq_pre],outputs=[output])
    adam = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
    model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["mae", fmeasure])
    return model

# 数据预处理

In [5]:
data = pd.read_csv("./data/data_train.csv",header=None,sep='\t',encoding='gbk')
data.columns = ['id','class','content','label']
data['content'] = data['content'].fillna('空')
data.head()

,id,class,content,label
0,1,食品餐饮,买这套系统本来是用来做我们公司的公众号第三方平台代运营的，没想到还有app，而且每个都很方便...,2
1,2,食品餐饮,烤鸭还是不错的，别的菜没什么特殊的,1
2,3,食品餐饮,使用说明看不懂！不会用，很多操作没详细标明！,0
3,4,食品餐饮,越来越不好了，菜品也少了，服务也不及时。,0
4,5,食品餐饮,是在是不知道该吃什么好、就来了,1


停用词表

In [7]:
# -*- coding: utf-8 -*- 
stopwords = []
with open("./data/stopwords.txt",encoding='utf-8') as f:
    for line in f.readlines():
        line = line.strip()
        stopwords.append(line)

去停用词

In [8]:
def filter_map(arr):
    res = ""
    for c in arr:
        if c not in stopwords and c != ' ' and c != '\xa0'and c != '\n' and c != '\ufeff' and c != '\r':
            res += c
    return res

得到字符级

In [8]:
def get_char(arr):
    res = []
    for c in arr:
        res.append(c)
    return list(res)
data['char'] = data.content.map(lambda x: filter_map(x))
data['char'] = data.content.map(lambda x: get_char(x))

得到词语级

In [9]:
data['word'] = data['content'].apply(lambda x:list(jieba.cut(x)))
data.to_csv("./data/data.csv", index=None)

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/nt/gh88gm3j7xndfrl3xyqv6zy40000gn/T/jieba.cache
Loading model cost 0.687 seconds.
Prefix dict has been built succesfully.


读取文件

In [9]:
data_all = pd.read_csv('./data/data.csv')

#随机扰乱
data_all = shuffle(data_all,random_state=0)
data_all.head()

,id,class,content,label,word
10405,10406,食品餐饮,环境挺好的，地方很大，蛋糕挺好吃的，不错,2,"['环境', '挺', '好', '的', '，', '地方', '很大', '，', '蛋..."
37421,37422,金融服务,我一个人在外地上班，刚开始嘛也没有挣多少钱，朋友推荐我了这个网站，我用了一次，全款买了个电脑...,2,"['我', '一个', '人', '在', '外地', '上班', '，', '刚', '开..."
68427,68428,物流快递,马氏鼎好物流公司，还可以，我们公司发空运的货物流全部找他们，速度快，价格合理，关键是查货方便...,2,"['马氏鼎', '好', '物流', '公司', '，', '还', '可以', '，', ..."
80884,80885,物流快递,时效很快，服务很好，双清关派送到门，对得起这个价格，赞！,2,"['时效', '很快', '，', '服务', '很', '好', '，', '双清', '..."
25191,25192,旅游住宿,其实旅游网都差不多，途牛好一点的是售后，不像有些，交完钱他就是大爷，对你不管不问。去欧洲回来...,2,"['其实', '旅游网', '都', '差不多', '，', '途牛', '好', '一点'..."


In [10]:
 data = data_all[data_all['class'] != '金融服务']  #四个类别的数据集  调参用
#data = data_all # 五个类别的数据集  验证模型适用于不同领域的隶属度

In [8]:
line_char = []
for s in data["char"]:
    line_char.append(s)

In [11]:
line_word = []
for s in data['word']:
    line_word.append(s)

训练词向量  _size维度:50 100 200 300       _type:word char         _embedding:word2vec fasttext glove

In [10]:
def train_embedding(_size,_type,_embeding):
    if _embeding == 'word2vec':
        if _type == 'char':
            word2vec_model = Word2Vec(line_char, size=_size,min_count=1,window=5,workers=4, iter=15)
            word2vec_model.save('./embeding/word2vec_char_'+str(_size)+'.vector')
        else:
            word2vec_model = Word2Vec(line_word, size=_size,min_count=1,window=5,workers=4, iter=15)
            word2vec_model.save('./embeding/word2vec_word_'+str(_size)+'.vector')
    elif _embeding == 'fasttext':
        if _type == 'char':
            fasttext_model = FastText(line_char, size=_size,window=10,min_count=1,iter=15)
            fasttext_model.save('./embeding/fasttext_char_'+str(_size)+'.vector')
        else:
            fasttext_model = FastText(line_word, size=_size,window=10,min_count=1,iter=15)
            fasttext_model.save('./embeding/fasttext_word_'+str(_size)+'.vector')
    elif _embeding =='glove':
        if _type == 'char':
            corpus_model = Corpus()
            corpus_model.fit(line_char, window=10)
            glove = Glove(no_components=_size, learning_rate=0.05)
            glove.fit(corpus_model.matrix, epochs=10,no_threads=1, verbose=True)
            glove.add_dictionary(corpus_model.dictionary)
            glove.save('./embeding/glove_char_'+str(_size)+'.model')
            corpus_model.save('./embeding/glove_corpus_char_'+str(_size)+'.model')
        else:
            corpus_model = Corpus()
            corpus_model.fit(line_word, window=10)
            glove = Glove(no_components=_size, learning_rate=0.05)
            glove.fit(corpus_model.matrix, epochs=10,no_threads=1, verbose=True)
            glove.add_dictionary(corpus_model.dictionary)
            glove.save('./embeding/glove_word_'+str(_size)+'.model')
            corpus_model.save('./embeding/glove_corpus_word_'+str(_size)+'.model')

In [ ]:
train_embedding(100,'word','word2vec')

tokenizer编码

向量维度:embed_size   截取长度:maxlen  line:line_word  line_char

In [12]:
def convert_embedding(embed_size,_type,_embeding,line,maxlen):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(line)
    data['Discuss_seq'] = tokenizer.texts_to_sequences(line)
    word_index = tokenizer.word_index
    embedding_matrix = np.zeros((len(word_index) + 1, embed_size))
    if _embeding == 'word2vec':
        if _type == 'char':
            model = Word2Vec.load("./embeding/word2vec_char_"+str(embed_size)+".vector")
        else:
           # model = Word2Vec.load("./embeding/word2vec_word_"+str(embed_size)+".vector")
            model = Word2Vec.load("./embeding/word2vec_word_"+str(embed_size)+"_skip.txt")
    elif _embeding == 'fasttext':
        if _type == 'char':
            model = FastText.load('./embeding/fasttext_char_'+str(embed_size)+'.vector')
        else:
            model = FastText.load('./embeding/fasttext_word_'+str(embed_size)+'.vector')
    elif _embeding == 'glove':
        if _type == 'char':
            model = Glove.load('../vector/glove_char_'+str(embed_size)+'.model')
        else:
            model = Glove.load('../vector/glove_word_'+str(embed_size)+'.model')
    for word, i in word_index.items():
        try:
            embedding_vector = model[word]
            if embedding_vector is not None:
                embedding_matrix[i] = embedding_vector
        except:
            pass
    return word_index, embedding_matrix

In [18]:
#from gensim.models import word2vec
from gensim.models import KeyedVectors

def convert_embedding2(embed_size,line,maxlen):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(line)
    data['Discuss_seq'] = tokenizer.texts_to_sequences(line)
    word_index = tokenizer.word_index
    embedding_matrix = np.zeros((len(word_index) + 1, embed_size))
    model =   KeyedVectors.load_word2vec_format("./embeding/word2vec_word_"+str(embed_size)+"_skip.txt",binary=False)
    for word, i in word_index.items():
        try:
            embedding_vector = model[word]
            if embedding_vector is not None:
                embedding_matrix[i] = embedding_vector
        except:
            pass
    return word_index, embedding_matrix

In [19]:
embed_size = 300
maxlen = 100
word_index, embedding_matrix = convert_embedding2(embed_size,line_word,maxlen)

D:\python\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [20]:
import pickle
def save_file(obj,file):
    with open(file,'wb') as f:
        pickle.dump(obj, f)
def load_file(file):
    with open(file,'rb') as f:
        return pickle.load(f)

In [21]:
save_file(embedding_matrix,'./data/embedding_matrix_300d.pkl')

save_file(word_index, './data/word_index_300d.pkl')

In [30]:
data_train  = data[:int(len(data) * 0.9)]#训练集
data_test = data[int(len(data) * 0.9):]#测试集

训练模型

In [38]:
def train_model(_filters, _kernel, _neurons, _drop, epoch=1):
    early_stopping = EarlyStopping(monitor="val_loss", mode="min", patience=3)
    callbacks_list = [early_stopping]
    _data_train = shuffle(data_train)
    X_train = pad_sequences(_data_train.Discuss_seq,maxlen=maxlen)
    y_train = np_utils.to_categorical(_data_train.label.values)
    final_model = cnn_lstm(_filters, _kernel, _neurons, _drop)
    final_model.summary()
    history = final_model.fit(X_train, y_train,
            validation_split=0.1, #%10作为验证集
            batch_size=128, 
            epochs= epoch, 
            shuffle = True,
            callbacks=callbacks_list)
    return final_model

# 测试集结果

In [16]:
def predict(model):
    X_test = pad_sequences(data_test.Discuss_seq,maxlen=maxlen)
    preds = model.predict(X_test)
    result = []
    for i in preds:
        r = i.tolist().index(max(i.tolist()))
        result.append(r)
#     a = range(1,len(result)+1)
#     a = np.array(a)
#     df = pd.DataFrame({'ind':a,'pre':result})
#     df.to_csv('./data/result.csv',index = False,header=False)
    return preds, result

正向f1值计算

In [17]:
def f_score(y_true, y_pred):
    #res = []
    true_positives = pd.DataFrame(y_true + y_pred)['label'].value_counts()[4]
    predicted_positives = pd.DataFrame(y_pred)[0].value_counts()[2]
    p = true_positives / (predicted_positives + K.epsilon())
    
   # true_positives = pd.DataFrame(y_true + y_pred)['label'].value_counts()[4]
    possible_positives = y_true.value_counts()[2]
    r = true_positives / (possible_positives + K.epsilon())

    f1 = 2* (p * r) / ( p + r + K.epsilon())
    #res.append([p,r,f1])
    return [p,r,f1]

In [ ]:
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

def eval_(y_true,y_pred):
    p_pos = precision_score(y_true, y_pred, labels=[2], average = 'micro')
    p_neg = precision_score(y_true, y_pred, labels=[0], average='micro')

    r_pos = recall_score(y_true, y_pred, labels=[2], average='micro')
    r_neg = recall_score(y_true, y_pred, labels=[0], average='micro')

    f1_pos = f1_score(y_true, y_pred, labels=[2], average = 'micro')
    f1_neg = f1_score(y_true, y_pred, labels=[0], average = 'micro')

    pos_rate = 6560/(6560+1224)
    #neg_rate = 12240/(65598+12240)

    p_w = p_pos*pos_rate+ p_neg*(1-pos_rate)
    r_w = r_pos*pos_rate+ r_neg*(1-pos_rate)
    f1_w = f1_pos * pos_rate + f1_neg * (1 - pos_rate)
    return p_w,r_w,f1_w

In [18]:
def experiment(_repeats, _filters, _kernel, _neurons, _drop):
    evaluate_result = []
    sum_preds = np.zeros((len(data_test), 3))
    for i in range(_repeats):
        model = train_model(_filters, _kernel, _neurons, _drop)
        preds, result = predict(model)
        sum_preds += preds
        res = f_score(data_test.label,result)
        evaluate_result.append(res)
    sum_preds /= _repeats
    with open("./data/result.txt","w") as ipt:
        for line in sum_preds:
            for i in line:
                ipt.write(str(i)+" ")
            ipt.write('\n')
    return np.array(evaluate_result).mean(axis=0),sum_preds

调整参数 filters, kernel, neurons, drop

In [19]:
filters = [8,16,24,32,48,64,96,128,192,256,384,512]
def train_filters(variable):
    all_result = []
    for i in variable:
        error_result,sum_preds = experiment(2, i, 3, 128, 0.1)
        all_result.append([i,error_result])
    return all_result

In [20]:
kernel = [1,2,3,4,5]
def train_kernel(variable):
    all_result = []
    for i in variable:
        f1_result,sum_preds = experiment(2, 128, i, 128, 0.1)
        all_result.append([i,f1_result])
    return all_result

In [21]:
neurons = [8,16,24,32,48,64,96,128,192,256]
def train_filters(variable):
    all_result = []
    for i in variable:
        f1_result,sum_preds = experiment(2, 128, 3, i, 0.1)
        all_result.append([i,f1_result])
    return all_result

In [22]:
drop = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
def train_drop(variable):
    all_result = []
    for i in variable:
        f1_result,sum_preds = experiment(2, 128, 3, 128, i)
        all_result.append([i,f1_result])
    return all_result

In [ ]:
train_drop(drop)

In [27]:
f1_result,sum_preds = experiment(_repeats=1, _filters=256, _kernel=3, _neurons=128, _drop=0.1)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 100)               0         
_________________________________________________________________
embedding_3 (Embedding)      (None, 100, 100)          6125600   
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 100, 256)          77056     
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 50, 256)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               197120    
_________________________________________________________________
dropout_3 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 387       
Total para